#### Viernes 16 de mayo de 2025

## Reglas de Asociación. Algoritmo Apriori. Optimización de Inventario

In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

#### Contexto: En Farmacia Saludable, quieren reducir desperdicios analizando compras conjuntas:

In [4]:
# Transacciones

transacciones = [
    ['Analgésico', 'Jarabe', 'Curitas'],
    ['Termómetro', 'Jarabe'],
    ['Analgésico', 'Termómetro', 'Alcohol'],
    ['Jarabe', 'Alcohol', 'Curitas'],
    ['Analgésico', 'Jarabe'],
    ['Termómetro', 'Curitas'],
    ['Analgésico', 'Alcohol']
]

print("Dataset Original: \n")
for i, transaccion in enumerate(transacciones, 1):
    print(f"Transacción {i}: {transaccion}")

Dataset Original: 

Transacción 1: ['Analgésico', 'Jarabe', 'Curitas']
Transacción 2: ['Termómetro', 'Jarabe']
Transacción 3: ['Analgésico', 'Termómetro', 'Alcohol']
Transacción 4: ['Jarabe', 'Alcohol', 'Curitas']
Transacción 5: ['Analgésico', 'Jarabe']
Transacción 6: ['Termómetro', 'Curitas']
Transacción 7: ['Analgésico', 'Alcohol']


### Preprocesamiento de Datos

In [ ]:
te = TransactionEncoder()

# Matriz booleana donde True indica la presencia del producto en la transacción
matriz = te.fit(transacciones).transform(transacciones)

df = pd.DataFrame(matriz, columns=te.columns_)

print("\nDataset Transormado (One-Hot Encoding): \n")
print(df)


Dataset Transormado (One-Hot Encoding): 



,Alcohol,Analgésico,Curitas,Jarabe,Termómetro
0,False,True,True,True,False
1,False,False,False,True,True
2,True,True,False,False,True
3,True,False,True,True,False
4,False,True,False,True,False
5,False,False,True,False,True
6,True,True,False,False,False


### Algoritmo Apriori. Items Frecuentes

In [ ]:
items_frecuentes = apriori(df, min_support=0.2, use_colnames=True)

print("\nItems Frecuentes (Soporte >= 20%): \n")
print(items_frecuentes.sort_values('support', ascending=False))


Items Frecuentes (Soporte >= 20%): 



,support,itemsets
1,0.571429,(Analgésico)
3,0.571429,(Jarabe)
0,0.428571,(Alcohol)
2,0.428571,(Curitas)
4,0.428571,(Termómetro)
5,0.285714,"(Analgésico, Alcohol)"
6,0.285714,"(Analgésico, Jarabe)"
7,0.285714,"(Jarabe, Curitas)"


### Reglas de Asociación

In [ ]:
# Reglas con confianza mínima del 50%
reglas = association_rules(items_frecuentes, metric="confidence", min_threshold=0.5)

#Filtramos reglas con lift > 1
reglas = reglas[reglas['lift'] > 1]

print("\nReglas de Asociación (Confianza >= 50%):\n")
print(reglas[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('lift', ascending=False))


Reglas de Asociación (Confianza >= 50%):



,antecedents,consequents,support,confidence,lift
0,(Analgésico),(Alcohol),0.285714,0.500000,1.166667
1,(Alcohol),(Analgésico),0.285714,0.666667,1.166667
4,(Jarabe),(Curitas),0.285714,0.500000,1.166667
5,(Curitas),(Jarabe),0.285714,0.666667,1.166667


### Pregunta Clave

#### Si un cliente compra Jarabe, ¿qué otro producto tiene mayor probabilidad de comprar? 

In [38]:
# De acuerdo a las reglas generadas se tiene un 50% de probabilidad de comprar Curitas 

#### ¿Qué par de productos debería colocarse juntos en el estante? Justifica con lift.

In [39]:
# Alcohol y Analgésico, con una probabilidad del 66.66% de que al comprar Alcohol, se compre Analgésico.
# Esto significa que comprar Alcohol aumenta la probabilidad de comprar Analgésico en un 16.6% respecto a lo normal.
# Además:
# Curitas y jarabe, con una probabilidad del 66.66% de que al comprar curitas, se compre jarabe.
# Esto significa que comprar Curitas aumenta la probabilidad de comprar jarabe en un 16.6% respecto a lo normal.